# Machine Learning from Scratch : -Part 1- Decision Trees and Random Forests

## 1. Aims and Structure

This notebook will guide you through the implementation of Decision Trees and Random Forests from scratch. It is divided into three main sections:
  
> - **Utility Functions Section:**: Implements essential helper functions that support the core functionality of both algorithms.
> - **Decision Tree Section:**: Which contains the main class of a Decision Tree classifier model
> - **Random Forest Section:**: Implements an ensemble of Decision Trees to form a Random Forest model.
---
> **Note**: This implementation relies solely on the NumPy library. Please adhere to this standard.
---

## 2. Theoretical Aspects

> The Decision Tree Algorithm is an information-based model in machine learning used for both classification and regression tasks (Classification in this lab). It is constructed based on an **information metric**, which is why it is referred to as an information-based model.
> In a Decision Tree, the model is structured as a hierarchy of nodes and arcs. Three types of nodes can be distinguished:
>
>> - ***Root node***: The first node in the tree, containing the entire dataset.
>> - ***Interior nodes***: Intermediate nodes that help in decision-making but do not provide final classifications themselves.
>> - ***Leaf nodes***: Terminal nodes without children that serve as decision nodes, providing the final classification or regression output.

> ### Process
>The process invovle the following steps:
>
>1. Compute the information gain for each feature (**column**) in the dataset.
>2. Select the **feature** with the **highest** information gain.
>3. Split the dataset based on the **selected feature**, creating subsets for each **unique value** of that feature.
>4. Recursively repeat this process for each subset, considering only the **remaining features** at each step.

>### Node Structure
>
> Each node in the tree contains the following **attributes**:
>
>***
>- Dataset Sample: A subset of the dataset (for the root node, it contains the entire dataset), divided into **features** and **target** values (**vectors**).
>- Decision Label: The most **frequent class label** in the dataset sample.
>- Feature (**Column**): The feature used for splitting at that node.
>- Depth: The level of the node within the tree.
>- Subsets (**Children Nodes**): The resulting subsets after splitting, stored as a dictionary of nodes
***

>### Information gain metric

>As mentioned earlier, Decision Trees use the Information Gain metric to determine the best feature for splitting the dataset and constructing nodes and leaves. To implement this metric effectively, it is essential to first understand its formula, meaning, and working mechanism.

>Information Gain measures the amount of information obtained about the target variable when splitting the dataset based on a particular feature. It quantifies how much uncertainty (entropy) is **reduced** by considering that **feature**.

>The uncertainty (or impurity) in the dataset is measured using Entropy, which is calculated as the weighted sum of the logarithms of the probabilities of each possible outcome (**class label**). In other words, entropy quantifies how impure or random the dataset is before and after a split.

>The entropy formula is given by:
>
> H(t) = − ∑ (P(t = i) × logs(P(t = i)))

---

Now, let’s move on to the fun part—**coding!** 😁🚀
---


# 3. Implementation

> ## Utility Functions

> The first method is **entropy** which is given the selected feature as a list (numpy array) named *column*.

> - From *column*, get two arrays; the uniques **values** (possible values) and their **counts** (or frequency)
> - Calculate the probability for each value in values (careful which array to use 😉)
> - Calculate the log2 probabilities for each probability

In [1]:
import numpy as np

def entropy(column):
    values, counts = np.unique(column, return_counts=True)  # Get unique values and their counts
    probabilities = counts / counts.sum()  # Compute probabilities
    log2probabilities = np.log2(probabilities)  # Compute log base 2 of probabilities

    return -np.dot(probabilities, log2probabilities)  # Compute entropy



> Now you need to calculate the remainder entropy (weighted entropy) after using the selected feature for splitting
> The method is given two arrays; column (feature) and y (target)
> - From *column*, get two arrays; the uniques **values** (possible values) and their **counts** (or frequency)
> - Calculate the **weights** which are the **probabilities** of each value in the column list
> - Now for each value, get its **entropy**

In [2]:
# TODO: this method needs to return the reminder part of the information gain formula

def rem(column, y):
    # Get unique values in the column and their counts
    values, counts = np.unique(column, return_counts=True)
    # Calculate the weights as the probabilities of each unique value
    weights = counts / len(column)
    
    # For each unique value, compute the entropy of the subset of y corresponding to that value
    entropies = []
    for val in values:
        # Create a mask to filter y where column equals the current value
        mask = (column == val)
        subset_y = y[mask]
        entropies.append(entropy(subset_y))
    
    # Return the weighted average of the entropies (remainder)
    return np.dot(weights, entropies)


> Now you just need to substract the **remainder entropy** (using the column) from the **total entropy** in the given node (using the target array).


In [3]:
# TODO: this method implements the information gain formula
def information_gain(column, y):
    return entropy(y) - rem(column, y)


> This function is used to get the most frequent label within a given list X

In [4]:
def mode(x):
    values, counts = np.unique(x, return_counts=True)  # Get unique elements and their frequencies
    index = np.argmax(counts)  # Find the index of the highest count
    return values[index]       # Return the element with the highest frequency


> This function evaluates the performance of our model (How many correct predictions out of the total number of observations).

In [36]:
def accuracy(y_pred, y_true):
    # Convert to numpy arrays in case they aren't already
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    
    # Calculate the fraction of predictions that are correct
    return np.mean(y_pred == y_true)


> ## Decision Tree

> Our Decision Tree will consist of two main classes; **DecisionTree Classifier** which is the main class responsible for constructing the tree, fitting (training) and predicting (classifying). The **Node Class** represents individual nodes within the tree.
> Each DecisionTree classifier object will build a tree composed of multiple Node objects.

> ### Node Class
First, we define a Node class to represent each node in the Decision Tree.

>> Refer to **Node Structure** subsection

In [7]:
class Node:
    def __init__(self, column=None, label=None, data=None, target=None):
        self.column = column    # The feature column used for splitting at this node
        self.label = label      # The label for leaf nodes
        self.data = data        # The data at this node
        self.target = target    # The target values corresponding to the data
        self.children = {}      # Initialize children as an empty dictionary

    def __str__(self):
        if not self.children:
            return f"[ Label: {self.label} ]"
        return str(self.children)

    def __repr__(self):
        return self.__str__()


> ### Decision Tree Classifier Class
> The DecisionTreeClassifier class requires a max_depth parameter, which defines the maximum depth the tree can reach. This helps control overfitting by limiting how deep the tree can grow.

> - The ***fit method*** Constructs the decision tree, starting from the **root node** and *recursively* growing branches until reaching the **leaf nodes**.

> - The **predict method** Classifies new instances by traversing the tree from the root node down to a leaf node. The path is determined by comparing feature values at each node.

> - The **find_best_split method** Uses the *information_gain* utility function to evaluate each feature and select the one that provides the best split.

In [18]:
import numpy as np
import pandas as pd


# Decision Tree class
class DecisionTree:
    def __init__(self, max_depth=10):
        self.max_depth = max_depth  # Set maximum depth for the tree

    def fit(self, X, y):
        # Initialize the root node with all data and targets.
        self.root = Node(data=X, target=y)
        # Stack holds tuples of (node, current_depth)
        stack = [(self.root, 0)]
        
        while stack:
            node, depth = stack.pop()  # Get the next node and its depth
            # Set the node's label to the mode of its target values
            node.label = mode(node.target)
            
            # If no more data to split on or maximum depth reached, skip splitting.
            if 0 in node.data.shape or depth >= self.max_depth:
                continue
            
            # Find the best feature to split on
            best_split = self.__find_best_split(node.data, node.target)
            node.column = best_split
            
            # Get the column data for the selected feature
            column = node.data[:, best_split]
            # Get unique values of the selected feature
            values = np.unique(column)
            
            # For each unique value, create a child node
            for value in values:
                mask = (column == value)
                # Remove the best_split column from the child's data
                child_data = np.delete(node.data[mask], best_split, axis=1)
                child_target = node.target[mask]
                
                # If no data remains after the split, create an empty child node with parent's label
                if 0 in child_data.shape:
                    child = Node(data=child_data, target=child_target)
                    child.label = node.label
                else:
                    child = Node(data=child_data, target=child_target)
                    child.label = mode(child_target)
                    # Append child node with increased depth
                    stack.append((child, depth + 1))
                
                # Set the child node corresponding to the feature value
                node.children[value] = child

    def predict(self, X):
        # Apply the private __predict method along axis=1 (row-wise)
        return np.apply_along_axis(self.__predict, axis=1, arr=X)

    def __predict(self, x):
        x = x.copy()  # Create a copy of x to avoid modifying the original input
        current_node = self.root  # Start traversal at the root
        
        # Traverse the tree until reaching a leaf node
        while current_node.children:
            col = current_node.column
            # Get the value from the input corresponding to the feature used for splitting
            value = x[col]
            # Remove the used feature from x for subsequent splits
            x = np.concatenate((x[:col], x[col+1:]), axis=0)
            
            # Get the corresponding child node for the value
            temp = current_node.children.get(value, None)
            # If no child node exists for that value, return the current node's label
            if temp is None:
                return current_node.label
            # Otherwise, continue traversing down the tree
            current_node = temp
            
        return current_node.label

    def __find_best_split(self, X, y):
        # Define a lambda that computes information gain for a column
        f = lambda col: information_gain(col, y)
        # Apply the lambda function to each column (feature) of X
        gains = np.apply_along_axis(f, axis=0, arr=X)
        # Return the index of the feature with the maximum information gain
        return gains.argmax()


## Question:

What is the difference in functionality between the **predict** and the **__predict** methods ? n/

The predict method is the public interface for making predictions on an entire dataset (or multiple samples), while the __predict method is a helper (private) method that makes a prediction for a single instance (a single row of features).

---
Now, we create our dataset, preprocess it, and then use our DecisionTreeClassifier to classify its instances.
---

In [19]:

data = {
    "ID": [1, 2, 3, 4, 5, 6, 7],
    "STREAM": [False, True, True, False, False, True, True],
    "SLOPE": ["steep", "moderate", "steep", "steep", "flat", "steep", "steep"],
    "ELEVATION": ["high", "low", "medium", "medium", "high", "highest", "high"],
    # target column:
    "VEGETATION": ["chapparal", "riparian", "riparian", "chapparal", "conifer", "conifer", "chapparal"]
}

df = pd.DataFrame(data)
df.head

<bound method NDFrame.head of    ID  STREAM     SLOPE ELEVATION VEGETATION
0   1   False     steep      high  chapparal
1   2    True  moderate       low   riparian
2   3    True     steep    medium   riparian
3   4   False     steep    medium  chapparal
4   5   False      flat      high    conifer
5   6    True     steep   highest    conifer
6   7    True     steep      high  chapparal>

In [20]:
X = df.drop(columns=["VEGETATION"])
y = df["VEGETATION"]


In [21]:
df = df.drop(columns=["ID"])


In [22]:
columns = X.columns.tolist()

df_original = df.copy()

df = df.to_numpy()

X = X.to_numpy()


In [23]:
clf = DecisionTree(max_depth=5)  # declare a Decision Tree Classifier with a max depth of 5

clf.fit(X, y)  # fit the classifier with X and y

print(clf.root)  # print the classifier's root node


{1: {False: {'steep': {'high': [ Label: chapparal ]}}}, 2: {True: {'moderate': {'low': [ Label: riparian ]}}}, 3: {True: {'steep': {'medium': [ Label: riparian ]}}}, 4: {False: {'steep': {'medium': [ Label: chapparal ]}}}, 5: {False: {'flat': {'high': [ Label: conifer ]}}}, 6: {True: {'steep': {'highest': [ Label: conifer ]}}}, 7: {True: {'steep': {'high': [ Label: chapparal ]}}}}


In [24]:
y_pred = clf.predict(X)  # predict using X

print(accuracy(y_pred, y))  # print the accuracy using the accuracy function


1.0


---

Curious about how our classifier compares to Scikit-Learn's DecisionTreeClassifier? Let's import it and compare the results!
---

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder

# Encode categorical features
encoder = OrdinalEncoder()
X_encoded = encoder.fit_transform(df_original.drop(columns=["VEGETATION"]))

# Encode target variable
y_encoded = pd.factorize(df_original["VEGETATION"])[0]

# Train a DecisionTreeClassifier
clf_sklearn = DecisionTreeClassifier(max_depth=5, random_state=42)
clf_sklearn.fit(X_encoded, y_encoded)

# Print the decision tree structure
from sklearn.tree import export_text
print(export_text(clf_sklearn, feature_names=df_original.drop(columns=["VEGETATION"]).columns.tolist()))


|--- ELEVATION <= 1.50
|   |--- ELEVATION <= 0.50
|   |   |--- SLOPE <= 1.00
|   |   |   |--- class: 2
|   |   |--- SLOPE >  1.00
|   |   |   |--- class: 0
|   |--- ELEVATION >  0.50
|   |   |--- class: 2
|--- ELEVATION >  1.50
|   |--- STREAM <= 0.50
|   |   |--- class: 0
|   |--- STREAM >  0.50
|   |   |--- class: 1



In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder

# Convert X back to a DataFrame for proper encoding
X_df = pd.DataFrame(X, columns=columns)

# Apply Ordinal Encoding to categorical features
encoder = OrdinalEncoder()
X_df[['STREAM', 'SLOPE', 'ELEVATION']] = encoder.fit_transform(X_df[['STREAM', 'SLOPE', 'ELEVATION']])

# Convert back to NumPy for training
X_encoded = X_df.to_numpy()

# Fit the DecisionTreeClassifier
clf_sklearn = DecisionTreeClassifier(max_depth=5, random_state=42)
clf_sklearn.fit(X_encoded, y)

# Print the trained decision tree structure
from sklearn.tree import export_text
print(export_text(clf_sklearn, feature_names=columns))


|--- ID <= 2.50
|   |--- ID <= 0.50
|   |   |--- class: chapparal
|   |--- ID >  0.50
|   |   |--- class: riparian
|--- ID >  2.50
|   |--- ELEVATION <= 2.00
|   |   |--- ID <= 5.50
|   |   |   |--- class: conifer
|   |   |--- ID >  5.50
|   |   |   |--- class: chapparal
|   |--- ELEVATION >  2.00
|   |   |--- class: chapparal



In [37]:
# Compute accuracy using your custom function
acc = accuracy(y_pred, y)
print(f"Accuracy: {acc:.2f}")



Accuracy: 0.43


## Random Forest

Random Forest is an ensemble learning method that builds multiple decision trees and combines their outputs to improve accuracy and reduce overfitting. It follows the principle of Bootstrap Aggregation (Bagging) and Feature Randomization.


*   **Bootstrap Sampling:** Each decision tree is trained on a random subset of data (rows are sampled with replacement).

*   **Feature Subsampling:** Each tree is trained using only a random subset of features, reducing correlation between trees.

*   **Majority Voting:** The final prediction is determined by taking the most common prediction across all trees.




In [40]:
import numpy as np

class RandomForestClassifier:
    def __init__(self, n_estimators=10, max_depth=5, max_features=5):
        self.n_estimators = n_estimators  # Number of decision trees
        self.max_depth = max_depth  # Max depth for each decision tree
        self.max_features = max_features  # Max number of features per tree
        
        self.estimators = []  # List of decision tree classifiers
        self.selected_features = []  # List of selected feature indices for each tree

    def fit(self, X, y):
        # Ensure max_features does not exceed number of actual features
        self.max_features = min(self.max_features, X.shape[1])

        # Ensure n_estimators does not exceed 2^max_features
        self.n_estimators = min(self.n_estimators, 2 ** self.max_features)

        # Initialize decision trees
        self.estimators = [DecisionTree(max_depth=self.max_depth) for _ in range(self.n_estimators)]
        
        for i in range(self.n_estimators):
            # Sample rows with replacement (bootstrap sampling)
            row_sample_idx = np.random.choice(X.shape[0], size=X.shape[0], replace=True)
            # Sample columns without replacement
            col_sample_idx = np.random.choice(X.shape[1], size=self.max_features, replace=False)
            
            self.selected_features.append(col_sample_idx)  # Store selected features
            
            # Create new training set with selected features
            B = X[row_sample_idx][:, col_sample_idx]  
            B_labels = y[row_sample_idx]

            # Fit the decision tree on the sampled data
            self.estimators[i].fit(B, B_labels)

    def predict(self, X):
        predictions = []  # List to store predictions from all trees
        
        for estimator, features in zip(self.estimators, self.selected_features):
            # Predict using the current decision tree on the selected features
            pred = estimator.predict(X[:, features])
            predictions.append(pred)
        
        # Take the mode (majority vote) across all trees for each sample
        return np.apply_along_axis(mode, axis=0, arr=np.array(predictions))

rf = RandomForestClassifier(n_estimators=10, max_depth=5, max_features=5)
rf.fit(X, y)  # Train the random forest
y_pred = rf.predict(X)  # Make predictions

acc = accuracy(y_pred, y)  # Compute accuracy
print(f"Accuracy: {acc:.8f}")  # Print accuracy



Accuracy: 0.85714286


---
We will proceed into testing Random Forest implementation
---

### Data Preprocessing

In [41]:
data = {
    "ID": [1, 2, 3, 4, 5, 6, 7],
    "STREAM": [False, True, True, False, False, True, True],
    "SLOPE": ["steep", "moderate", "steep", "steep", "flat", "steep", "steep"],
    "ELEVATION": ["high", "low", "medium", "medium", "high", "highest", "high"],
    # Target column:
    "VEGETATION": ["chapparal", "riparian", "riparian", "chapparal", "conifer", "conifer", "chapparal"]
}

df = pd.DataFrame(data)
df.head

<bound method NDFrame.head of    ID  STREAM     SLOPE ELEVATION VEGETATION
0   1   False     steep      high  chapparal
1   2    True  moderate       low   riparian
2   3    True     steep    medium   riparian
3   4   False     steep    medium  chapparal
4   5   False      flat      high    conifer
5   6    True     steep   highest    conifer
6   7    True     steep      high  chapparal>

In [ ]:
X = df.drop(columns=["VEGETATION"])
y = df["VEGETATION"]


In [43]:
X = X[:, 1:]  # Removes the first column


In [44]:
# TODO: Get columns names' list
columns = df.columns.tolist()

# TODO: Create a copy of the original DataFrame
df_original = df.copy()

# TODO: Convert df to a NumPy array
df = df.to_numpy()

# TODO: Convert X to a NumPy array (excluding the target column)
X = df[:, :-1]  # Assuming "VEGETATION" is the last column


### Fitting and Predicting

In [45]:
# TODO: Create a RandomForestClassifier Object
clf = RandomForestClassifier(n_estimators=10, max_depth=5, max_features=3)

# TODO: Fit the classifier
clf.fit(X, y)

# TODO: Predict using X
y_pred = clf.predict(X)

# TODO: Print the accuracy (using the accuracy function that you worked on before)
acc = accuracy(y_pred, y)
print(f"Accuracy: {acc:.2f}")


Accuracy: 1.00


# Extra

> If you examine our DecisionTree classifier, you'll notice that it only considers **categorical features** when splitting nodes.
### Task

> The Elevation attribute is a **continuous variable** with the following values: "ELEVATION": [3900, 300, 1500, 1200, 4450, 5000, 3000]. Modify the classifier to properly handle continuous features by implementing an appropriate splitting strategy.
